In [1]:
import pandas as pd
import os
from pathlib import Path
df=pd.read_excel("D:/College\Machine Learning/heckhers credit card default prediction/default of credit card clients.xls")
df=df.drop(index=0)
df=df.drop(df.columns[0],axis=1)
print(df.shape)
df.head(5)
df.dropna(inplace=True)

(30000, 24)


In [2]:
#alter the dataframe

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df=df_scaled=pd.DataFrame(scaler.fit_transform(df),columns=df.columns)
df_scaled.head(12)
df=df.sort_values(by=['Y'])

# df = df.tail(13200)
df=df.sample(frac=1)

X = df.drop('Y',1)
y = df['Y']
df=df.drop(df.columns[-1], axis=1)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

import numpy as np
X_train = np.reshape(X_train.values,(len(X_train),(X_train.shape[1]),1))
X_test = np.reshape(X_test.values,(len(X_test),(X_train.shape[1]),1))

In [4]:
print("Old Number of Features")
print(X_train.shape[1])

Old Number of Features
23


In [5]:
# Reduce the number of feature given the p-value

import statsmodels.regression.linear_model as sm
def backwardElimination(x, Y, sl):
    x = np.array(x, dtype=float)
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    
    regressor_OLS.summary()
    return x

SL = 0.05
data_modeled = backwardElimination(X, y,SL)

In [6]:
print("Number of Features after Dimentionality Redection")
print(data_modeled.shape[1])

Number of Features after Dimentionality Redection
15


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_modeled,y,test_size=0.2)

import numpy as np
X_train = np.reshape(X_train,(len(X_train),data_modeled.shape[1],1))
X_test = np.reshape(X_test,(len(X_test),data_modeled.shape[1],1))

In [11]:
X_train.shape

(24000, 15, 1)

In [12]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold

poly = PolynomialFeatures(2)

temp_x_train = np.reshape(X_train,(len(X_train),15))
temp_y_train = np.reshape(y_train,(len(y_train)))

temp_x_test = np.reshape(X_test,(len(X_test),15))
temp_y_test = np.reshape(y_test,(len(y_test)))

sel = VarianceThreshold(threshold=.01)

sel.fit(temp_x_train)


temp_x_train  

reg = LogisticRegression().fit(poly.fit_transform(temp_x_train), temp_y_train)


reg.intercept_
y_predict= reg.predict(poly.fit_transform(temp_x_test))
print("Logistic Regression using sklearn")
print("Training Accuracy: "+ str(reg.score(poly.fit_transform(temp_x_train),temp_y_train)))
print("Testing Accuracy: "+ str(reg.score(poly.fit_transform(temp_x_test),temp_y_test)))


from sklearn.metrics import r2_score
print("R2 score: " + str(r2_score(temp_y_test,y_predict)))

Logistic Regression using sklearn
Training Accuracy: 0.8116666666666666
Testing Accuracy: 0.8135
R2 score: -0.10814479824717038


In [13]:
X_train.shape

(24000, 15, 1)